In [8]:
!pip install keras_applications
!pip install keras==2.1.5

  Using cached Keras-2.1.5-py2.py3-none-any.whl (334 kB)
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-macos 2.12.0 requires keras<2.13,>=2.12.0, but you have keras 2.1.5 which is incompatible.


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
import numpy as np


In [10]:
 vgg_model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB

94694792/94694792 [==============================] - 31s 0us/step


In [53]:
import cv2
def extract_face_embeddings(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    x = tf.keras.preprocessing.image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x, version=2)
    features = vgg_model.predict(x)
    return features[0]
def extract_face(image):
    # Load the Haar Cascade for face detection
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    # Detect the faces in the image
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5)
    
    # Extract the first face (assuming there's only one)
    for (x, y, w, h) in faces:
        face = image[y:y+h, x:x+w]
        return face
    
    # If no faces were detected, return a placeholder image
    return np.zeros_like(image)



In [58]:
import os
import numpy as np
import cv2
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from sqlalchemy import Column, Integer, LargeBinary, create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

# Define the database engine and session
engine = create_engine('postgresql://postgres:@localhost:5430/faceio')
Base = declarative_base()

class FaceEmbedding(Base):
    __tablename__ = 'face_embeddings'

    empl_no = Column(Integer, primary_key=True)
    embedding = Column(LargeBinary)

Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()

# Define the directory containing the face images
image_dir = '/Users/ds_learner16/Documents/Face.io/photos/train'

# Load the VGGFace model
model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')

# Loop through the directories in the image directory
for dir_name in os.listdir(image_dir):
    if not os.path.isdir(os.path.join(image_dir, dir_name)):
        continue
    
    # Extract the person's name from the directory name
    empl_no = dir_name
    
    # Load and encode all face images for this person
    embeddings = []
    for file_name in os.listdir(os.path.join(image_dir, dir_name)):
        if not file_name.endswith('.jpg'):
            continue
        
        # Load the image file and extract the face
        image_path = os.path.join(image_dir, dir_name, file_name)
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        x = cv2.resize(img, (224, 224))
        x = preprocess_input(x.astype('float64'), version=2)
    
        
        # Encode the face using the VGGFace model
        embedding = model.predict(np.expand_dims(x, axis=0))[0]
        embeddings.append(embedding)
    
    # Compute the average face embedding and add it to the database
    if embeddings:
        avg_embedding = np.mean(embeddings, axis=0)
        new_embedding = FaceEmbedding(empl_no=empl_no, embedding=avg_embedding.tobytes())
        session.add(new_embedding)
    
session.commit()


1/1 [==============================] - 0s 22ms/step


In [79]:
!pip install dlib

  Using cached dlib-19.24.1.tar.gz (3.2 MB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [7 lines of output]
      running bdist_wheel
      running build
      running build_py
      running build_ext
      
      ERROR: CMake must be installed to build dlib
      
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for dlib
  Running setup.py clean for dlib
Failed to build dlib
  Running setup.py install for dlib ... error
  error: subprocess-exited-with-error
  
  × Running setup.py install for dlib did not run successfully.
  │ exit code: 1
  ╰─> [9 lines of output]
      running install
      /Users/ds_learner16/anaconda3/lib/python3.10/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other sta

In [19]:
!pip install tensorflow-macos

  Using cached keras-2.12.0-py2.py3-none-any.whl (1.7 MB)
  Attempting uninstall: keras
    Found existing installation: Keras 2.1.5
    Uninstalling Keras-2.1.5:
      Successfully uninstalled Keras-2.1.5


In [59]:
def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    similarity = dot_product / (norm_a * norm_b)
    return similarity

In [76]:
# Load the target face image and extract the face
target_image_path = '/Users/ds_learner16/Documents/Face.io/photos/train/0757/0757_test_06.jpg'
target_img = cv2.imread(target_image_path)
target_img = cv2.cvtColor(target_img, cv2.COLOR_BGR2RGB)
target_face = extract_face(target_img) # Replace this with your own face detection code

# Encode the target face using the same VGGFace model and preprocessing steps as used to generate the embeddings
target_embedding = model.predict(np.expand_dims(target_face, axis=0))[0]

# Compute the similarity between the target face embedding and the embeddings in the database
embeddings = session.query(FaceEmbedding).all()
similarity_scores = []
for embedding in embeddings:
    db_embedding = np.frombuffer(embedding.embedding, dtype=np.float32)
    similarity_scores.append(cosine_similarity(target_embedding, db_embedding))

# Retrieve the person's name associated with the embedding with the highest similarity score based on the order
similarity_scores = np.array(similarity_scores)
max_similarity_idx = np.argsort(-similarity_scores)[0] # Sort in descending order and retrieve the first index
person_embedding = session.query(FaceEmbedding).get(int(max_similarity_idx))
if person_embedding is not None:
    person_empl_no = person_embedding.empl_no
    print(f"Recognized person: {person_empl_no}")
else:
    print("No matching person found in the database.")


1/1 [==============================] - 0s 108ms/step
No matching person found in the database.


/var/folders/7m/m8d42_wx6_s2mq4bkrbnj5_h0000gn/T/ipykernel_14418/2872468698.py:5: RuntimeWarning: invalid value encountered in float_scalars
  similarity = dot_product / (norm_a * norm_b)


In [78]:
# Initialize the face detection cascade classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize the video capture
cap = cv2.VideoCapture(0)

while True:
    # Capture a frame from the video
    ret, frame = cap.read()

    # Convert the color space from BGR to RGB
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect and extract the face from the frame
    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        face = cv2.resize(face, (224, 224))
        face = cv2.cvtColor(face, cv2.COLOR_RGB2BGR)
        face_embedding = model.predict(np.expand_dims(target_face, axis=0))[0]
        similarity_scores = []
        for embedding in session.query(FaceEmbedding):
            db_embedding = np.frombuffer(embedding.embedding, dtype=np.float32)
            similarity_scores.append(cosine_similarity(face_embedding, db_embedding))
        similarity_scores = np.array(similarity_scores)
        max_similarity_idx = np.argsort(-similarity_scores)[0]
        person_embedding = session.query(FaceEmbedding).get(int(max_similarity_idx))
        if person_embedding is not None:
            person_empl_no = person_embedding.empl_no
            cv2.putText(frame, f"Recognized person: {person_empl_no}", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
        else:
            cv2.putText(frame, "No matching person found in the database", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)

    # Show the resulting frame
    cv2.imshow('Face Recognition', frame)

    # Break the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close all windows
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 38ms/step


/var/folders/7m/m8d42_wx6_s2mq4bkrbnj5_h0000gn/T/ipykernel_14418/2872468698.py:5: RuntimeWarning: invalid value encountered in float_scalars
  similarity = dot_product / (norm_a * norm_b)


1/1 [==============================] - 0s 28ms/step


KeyboardInterrupt: 